# Module

In [1]:
import argparse
from tqdm import tqdm_notebook as tq
import tqdm
import os, time, math, copy
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from collections import namedtuple
import matplotlib.pyplot as plt
import random

torch.set_printoptions(precision=8, linewidth=50000)
import warnings
warnings.filterwarnings(action='ignore')

# Print Colors

In [2]:
BLACK	= '\033[30m'
RED		= '\033[31m'
GREEN	= '\033[32m'
YELLOW	= '\033[33m'
BLUE	= '\033[34m'
MAGENTA	= '\033[35m'
CYAN	= '\033[36m'
RESET	= '\033[0m'
SEL		= '\033[7m'

In [3]:
class	fxp:
	def	__init__(self, bIn, iBWF):
		self.iFullBW	= len(bIn)
		self.iIntgBW	= self.iFullBW - iBWF
		self.bSign		= bIn[0]
		self.bIntg		= bIn[:self.iIntgBW]
		self.bFrac		= bIn[self.iIntgBW:]
		self.fFull		= 0
		try:
			for idx, bit in enumerate(bIn):
				if	idx == 0:
					self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
				else:
					self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
		except:
			print(bIn)
		self.dispFull	= RED + self.bIntg + BLUE + self.bFrac + RESET
		return

In [4]:
class	flp2fix:
	def	__init__(self, fIn, iBW, iBWF):
		self.fMin		= - 2 ** (iBW - iBWF - 1)
		self.fMax		= (2 ** (iBW-1) - 1) * (2 ** -iBWF)
		self.fResol		= 2 ** -iBWF
		if fIn < self.fMin or fIn > self.fMax:
			print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
		self.iBW		= iBW
		self.iBWI		= iBW - iBWF
		self.iBWF		= iBWF

		self.iFLP2INT	= abs(int(fIn * 2 ** iBWF))
		if fIn < 0:
			self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

		if fIn >= 0:
			self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
		else:
			self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
			if len(self.bFull) > iBW:
				self.bFull = '0' * iBW

		self.cssFxp		= fxp(self.bFull, self.iBWF)
		self.bSign		= self.cssFxp.bSign
		self.bIntg		= self.cssFxp.bIntg
		self.bFrac		= self.cssFxp.bFrac
		self.fFull		= self.cssFxp.fFull
		return

In [5]:
def	flp2fixTensor(fIn, iBW, iBWF):
	fMin = - 2 ** (iBW - iBWF - 1)
	fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
	fList = []
	for aTensor in fIn.view(-1):
		fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
	return torch.tensor(fList).view(fIn.size())

# User Define Variables

In [6]:
data_path = '~/dataset'

In [30]:
parser = argparse.ArgumentParser(description='PyTorch for MNIST dataset')
parser.add_argument('--device', type=str, default='cpu', help='Device')
parser.add_argument('--shuffle', action='store_true', default=False, help='enables data shuffle')
parser.add_argument('--dataset', type=str, default='mnist', help='training dataset')
parser.add_argument('--data_path', type=str, default=data_path, help='path to MNIST')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs', type=int, default=10, help='number of epochs to train')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--optimizer', type=str, default='adam', help='optimizer')
parser.add_argument('--loss_func', type=str, default='cel', help='optimizer')
parser.add_argument('--quant_opt', type=str, default='asym', help='Type of Quantization')
parser.add_argument('--full_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=3, help='Number of Quantization Bits')
parser.add_argument('--pretrained', type=bool, default=True, help='Pretrained Model')
parser.add_argument('--act_quant', type=bool, default=False, help='Activation Quantization')
parser.add_argument('--disp', type=bool, default=False, help='Display Model Information')
parser.add_argument('--bBW',type=int,default=4,help='bit number')
args = parser.parse_args(args=[])

# Preparing Data

In [8]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.device == 'cuda' else {}
if args.dataset == 'mnist':
	train_loader = torch.utils.data.DataLoader(
		dataset=datasets.MNIST(
			root=args.data_path,
			train=True,
			download=True,
			transform=transforms.ToTensor()
		),
		batch_size=args.batch_size,
		shuffle=args.shuffle,
		**kwargs
	)

	test_loader = torch.utils.data.DataLoader(
		dataset=datasets.MNIST(
			root=args.data_path,
			train=False,
			download=True,
			transform=transforms.ToTensor()
		),
		batch_size=args.batch_size,
		shuffle=args.shuffle,
		**kwargs
	)

# Build Model

In [9]:
class MLP(nn.Module):
	def __init__(self):
		super(MLP, self).__init__()
		self.flatten = nn.Flatten()
		self.fc1 = nn.Linear(28*28, 16)
		self.relu1 = nn.ReLU()
		self.fc2 = nn.Linear(16, 16)
		self.relu2 = nn.ReLU()
		self.fc3 = nn.Linear(16, 10)
		
	def forward(self, x):
		x = self.flatten(x)
		x = self.fc1(x)
		x = self.relu1(x)
		x = self.fc2(x)
		x = self.relu2(x)
		logits = self.fc3(x)
		return logits

In [10]:
def genOptimizer(model, args):
	if args.optimizer == 'sgd':
		optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
	if args.optimizer == 'adam':
		optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
	return optimizer

def genLossFunc(args):
	if args.loss_func == 'cel':
		loss_func = nn.CrossEntropyLoss()
	return loss_func

In [11]:
def train(train_loader, model, epoch, args):
	model.train()
	loss_func = genLossFunc(args)
	optimizer = genOptimizer(model, args)
	max_batch_index = int(np.floor(len(train_loader.dataset)/args.batch_size))
	running_loss = 0
	for batch_index, (image, label) in enumerate(tq(train_loader, desc='Train', leave=False)):
		image, label = image.to(args.device), label.to(args.device)
		pred = model(image)
		loss = loss_func(pred, label)
		running_loss += loss.item()#*image.size(0)

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
				
	print(f'Epoch {epoch+1:<3d}: Avg. Loss: {running_loss/len(train_loader.dataset):.4f}', end = '\t')

In [12]:
def test(test_loader, model, args):
	model.eval()
	with torch.no_grad():
		loss_func = genLossFunc(args)
		loss, correct = 0, 0
# 		for batch_index, (image, label) in enumerate(tq(test_loader, desc='Test', leave=False)):
		for batch_index, (image, label) in enumerate(test_loader):
			image, label = image.to(args.device), label.to(args.device)
			pred = model(image)
			loss += loss_func(pred, label).item()#*image.size(0)
			correct += (pred.argmax(1) == label).type(torch.int).sum().item()
	loss /= len(test_loader.dataset)
	correct_rate = 100 * correct / len(test_loader.dataset)
	print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({correct_rate:>.1f}%)')

In [13]:
def main(model):
	for epoch in range(args.epochs):
		train(train_loader, model, epoch, args)
		test(test_loader, model, args)
	print("Done!")
	return model

In [14]:
if args.pretrained:
    if os.path.isfile('preTrainedModel.pth'):
        model = MLP().to(args.device)
        model.load_state_dict(torch.load('preTrainedModel.pth'))
        test(test_loader, model, args)
    else:
        model = main(MLP().to(args.device))
        torch.save(model.state_dict(), 'preTrainedModel.pth')
else:
	model = main(MLP().to(args.device))
	torch.save(model.state_dict(), 'preTrainedModel.pth')

Accuracy: 9524/10000 (95.2%)


In [15]:
print(model.fc1.weight)

Parameter containing:
tensor([[-0.02905918,  0.02385001,  0.00032381,  ..., -0.00330848,  0.03122507,  0.01620386],
        [ 0.02255142, -0.00389863,  0.00861061,  ...,  0.02175023, -0.03418417, -0.01858932],
        [-0.02584666,  0.00479941, -0.02506999,  ..., -0.02010476,  0.01693955,  0.01101075],
        ...,
        [ 0.03105099, -0.03084483,  0.00437470,  ..., -0.01286467,  0.03370811,  0.02605597],
        [-0.03491382, -0.00306286, -0.01362592,  ...,  0.02577536, -0.03532766, -0.01946842],
        [ 0.00877646,  0.00609850,  0.01847609,  ...,  0.01105409, -0.02793505,  0.00065047]], requires_grad=True)


# SNG

In [16]:
def Comp(a,lfsr,snum):
    for com in range(0,len(lfsr)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

In [17]:
def snum(a):
    if a > 0 :
        return '0'
    else :
        return '1'

In [18]:
class LFSR:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[2]),int(stream[3]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3

In [19]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else :
        iOut = '0'
    return iOut

In [20]:
def SNG(iIN):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    #lfsrlist = []
    
    lfsr = LFSR()
    
    for k in range(2**(args.bBW)): #lfsr number generating
        if k == 0:
            lNUM = lfsr.Random()
        else :
            lNUM = lfsr.Normal(lNUM)
        if (k == 2**(args.bBW)-1):
            lNUM = lfsr.Allzero()
        
        if len(lNUM) != len(bIN[-(args.frac_bits):]) :
            if len(lNUM) < len(bIN[-(args.frac_bits):]) :
                lNUM = lNUM + (len(bIN[-(args.frac_bits):])-len(lNUM))*'0'
            else :
                print("it can't work")
                return 0
        oAlist.append(Comp(bIN[-(args.frac_bits):],lNUM,sNUM)) #comparator of input a
        
    oAnum = oAlist.count('1')
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    return sA

In [21]:
def SNGTensor(fIn):
    sList = []
    for aTensor in fIn.view(-1):
        sList.append(SNG(float(aTensor)))
                     
    return torch.tensor(sList).view(fIn.size())

## Find max, min 

In [22]:
def findActMaxMin():
    SF = {}
    for i in range(3):
            max = torch.max(eval(f'act{i}'))
            min = torch.min(eval(f'act{i}'))
            exec(f"SF['act{i}']=torch.max(abs(max),abs(min)).item()")
    return SF

In [23]:
def findWeightMaxMin(model, args):
    SF = {}
    for name, data in model.named_parameters():
        if "fc" and "weight" in name :
            max = torch.max(data)
            min = torch.min(data)
            exec(f"SF['{name}']=torch.max(abs(max),abs(min)).item()")
    return SF

## Fixed model

In [24]:
def model2fix(model, args):
	for name, _ in model.named_parameters():
		exec(f'model.{name}.data = flp2fixTensor(model.{name}.data, {args.full_bits}, {args.frac_bits})')
	return model

In [25]:
def quantFixForward(model, x, args):
	cmodel = copy.deepcopy(model).to(args.device)
	with torch.no_grad():
		x = cmodel.flatten(x)
		act0 = flp2fixTensor(x, args.full_bits, args.frac_bits)
		
		act1 = model.relu1(cmodel.fc1(act0))
		act1 = flp2fixTensor(act1, args.full_bits, args.frac_bits)

		act2 = cmodel.relu2(cmodel.fc2(act1))
		act2 = flp2fixTensor(act2, args.full_bits, args.frac_bits)
		
		act3 = cmodel.fc3(act2)
		act3 = flp2fixTensor(act3, args.full_bits, args.frac_bits)
	return cmodel, act0, act1, act2, act3 

In [31]:
def testQuant(model, test_loader, args):
	qmodel = copy.deepcopy(model).to(args.device)
	qmodel = model2fix(qmodel, args)
	qmodel.eval()
	with torch.no_grad():
		loss_func = genLossFunc(args)
		loss, correct = 0, 0
		for batch_index, (image, label) in enumerate(tq(test_loader,desc='Test',leave=False)):
			image, label = image.to(args.device), label.to(args.device)
			qmodel, act0, act1, act2, act3  = quantFixForward(qmodel, image, args)
			y = act3
			loss += loss_func(y, label).item()#*image.size(0)
			correct += (y.argmax(1) == label).type(torch.int).sum().item()
	correct_rate = 100 * correct / len(test_loader.dataset)
	print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({correct_rate:>.1f}%) Loss: {loss/len(test_loader.dataset):.2f}')
	return qmodel, act0, act1, act2, act3

In [32]:
qmodel, act0, act1, act2, act3 = testQuant(model, test_loader, args)

Test:   0%|          | 0/157 [00:00<?, ?it/s]

(17.4375): Out of input range (15.875/-16) during flp -> fix converting 
(15.890625): Out of input range (15.875/-16) during flp -> fix converting 
(-16.75): Out of input range (15.875/-16) during flp -> fix converting 
1000b110
(17.15625): Out of input range (15.875/-16) during flp -> fix converting 
(17.84375): Out of input range (15.875/-16) during flp -> fix converting 
(16.453125): Out of input range (15.875/-16) during flp -> fix converting 
(15.984375): Out of input range (15.875/-16) during flp -> fix converting 
(17.421875): Out of input range (15.875/-16) during flp -> fix converting 
(17.0): Out of input range (15.875/-16) during flp -> fix converting 
(16.875): Out of input range (15.875/-16) during flp -> fix converting 
(16.3125): Out of input range (15.875/-16) during flp -> fix converting 
(18.109375): Out of input range (15.875/-16) during flp -> fix converting 
(16.46875): Out of input range (15.875/-16) during flp -> fix converting 
(18.46875): Out of input range (15

## apply SNG

In [33]:
def N2S(qmodel,act0,act1,act2,args):
    aSF = findActMaxMin()
    wSF = findWeightMaxMin(qmodel,args)
    
    a0 = SNGTensor(act0/aSF['act0'])
    a1 = SNGTesnor(act1/aSF['act1'])
    a2 = SNGTensor(act2/aSF['act2'])
    
    w0 = SNGTensor(qmodel.fc1.weight/wSF['fc1.weight'])
    w1 = SNGTensor(qmodel.fc2.weight/wSF['fc2.weight'])
    w2 = SNGTensor(qmodel.fc3.weight/wSF['fc3.weight'])
    
    return a0,a1,a2,w0,w1,w2

In [29]:
a0,a1,a2,w0,w1,w2 = N2S(qmodel,act0,act1,act2,args)

ValueError: too many dimensions 'str'